In [1]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [2]:
#visualise maze:
mazeSize = [4,12]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        

        
giftState = 47
            
print(giftState)
print(statePositions)

env = gym.make("CliffWalking-v0")

47
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]


In [40]:
def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 0

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 36:
        return [0]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(1)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(3)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(0)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(2)
        
    return possibleMoves

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]

        maxValue = max(qValues.values())
        minValue = min(qValues.values())
        count_max = sum(1 for value in qValues.values() if value == maxValue)
        count_min = sum(1 for value in qValues.values() if value == minValue)

        if count_max > 1 and count_max < len(possMoves):
            nextMove = random.choice([move for move in possMoves if qValues[move] != minValue])
        elif count_max == len(possMoves):
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def pathFound():
    currentState = 0
    for i in range(mazeSize[0]*mazeSize[1]-1):
        bestDirection = None
        if max(qTable_1[currentState]) != 0:
            bestDirection = qTable_1[currentState].index(max(qTable_1[currentState]))
        newState = 0
        if bestDirection == 0 and 0 in calcPossibleMoves(currentState):
            newState = currentState - 1
        elif bestDirection == 1 and 1 in calcPossibleMoves(currentState):
            newState = currentState + mazeSize[1]
        elif bestDirection == 2 and 2 in calcPossibleMoves(currentState):
            newState = currentState + 1
        elif bestDirection == 3 and 3 in calcPossibleMoves(currentState):
            newState = currentState - mazeSize[1]

        if newState == giftState:
            return True
        currentState = newState
    return False

def updateTable_monteCarlo(steps, totalReward):
    global qTable_1
    global currentState
    global direction
    minChangeInQ = 0
    didConverge = False
    avgReward = totalReward/len(steps)
    for i in range(len(steps)):
        updated = qTable_1[steps[i][0]][steps[i][1]] + alpha*(avgReward-qTable_1[steps[i][0]][steps[i][1]])
        changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
        if i == 0:
            minChangeInQ = changeInQ
        elif changeInQ < minChangeInQ and changeInQ != 0:
            minChangeInQ = changeInQ
        qTable_1[steps[i][0]][steps[i][1]] = updated
    if minChangeInQ < convergenceThresh and minChangeInQ > 0:
            if pathFound():
                didConverge = True
    return didConverge, minChangeInQ

In [41]:
epsilonValue = 0.35
alpha = 0.65
convergenceThresh = 0.01
revistPenalty = -0.25
totalTrials = 0
currentState = 36

In [42]:
def qLearning(maxEpisodes, currentTrial):
    global currentState
    global revisitPenalty
    global totalTrials
    global totalTime_start
    
    firstPath = []
    firstPathEpisode = []
    foundFirstPath = False
    
    currentState = 36
    currentEpisode = 1
    converged = False

    resetTable()
    env.reset()
    
    start_time = time.time()
    
    while currentEpisode <= maxEpisodes:
        if converged:
            break

        direction = nextStep(currentState)
        if not foundFirstPath:
            firstPath.append([currentState, direction])
        nextState, reward, terminated, truncated, info = env.step(direction)

        converged, changeInQ = updateTable_qLearning(direction, nextState, reward)

        if terminated or truncated or converged:
            observation, info = env.reset()
            currentState = 0
            if not converged:
                currentEpisode += 1
            if not foundFirstPath:
                firstPath = []


        if converged:
            end_time = time.time()
        
        time.sleep(0.001)
        clear_output(wait=True)
        if totalTrials != 0:
            print("Trial: " + 'Q-Learning, ' + 'trialType '+ str(trialType) + ', Trial ' + str(currentTrial) + "/" + str(totalTrials))
        print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
        print("Trial Time: " + str(round(time.time()-start_time, 3)) + " sec")
        if totalTrials != 0:
            print("Total Time: " + str(round((time.time()-totalTime_start)/60, 3)) + " min")
        print("Q-Table:")
        for i in range(len(qTable_1)):
            print(str(i) + ": " + str(qTable_1[i]))
        print("change in Q: " + str(changeInQ))
        
    duration = 0
    if converged:
        duration = end_time - start_time
        print(str(round(duration, 3)) + " seconds to converge")
    else:
        duration = None
        print("No convergence")
        
    return (currentEpisode-1), duration

In [43]:
def monteCarlo_everyVisit(maxEpisodes, currentTrial):
    global currentState
    global revisitPenalty
    global direction
    global totalTrials
    global totalTime_start
    
    firstPath = []
    firstPathEpisode = []
    foundFirstPath = False
    
    currentEpisode = 1
    converged = False

    episodeSteps = []
    totalEpisodeReward = 0

    resetTable()
    env.reset()
    
    start_time = time.time()
    
    while currentEpisode <= maxEpisodes:
        global currentState
        if converged:
            break

        direction = nextStep(currentState)
        if not foundFirstPath:
            firstPath.append([currentState, direction])
        episodeSteps.append([currentState, direction])
        nextState, reward, terminated, truncated, info = env.step(direction)

        totalEpisodeReward += reward
        currentState = nextState

        changeQ = 0
        if terminated or truncated:
            if nextState == giftState and not foundFirstPath:
                foundFirstPath = True
                firstPathEpisode = [currentEpisode, direction]
            observation, info = env.reset()
            converged, changeInQ = updateTable_monteCarlo(episodeSteps, totalEpisodeReward)
            changeQ = changeInQ
            episodeSteps = []
            currentState = 0
            totalEpisodeReward = 0
            currentEpisode += 1
            if not foundFirstPath:
                firstPath = []
            
        time.sleep(0.01)
        clear_output(wait=True)
        print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
        print("Trial Time: " + str(round(time.time()-start_time, 3)) + " sec")
        if totalTrials != 0:
            print("Total Time: " + str(round((time.time()-totalTime_start)/60, 3)) + " min")
        print("Q-Table:")
        for i in range(len(qTable_1)):
            print(str(i) + ": " + str(qTable_1[i]))
        print("change in Q: " + str(changeQ))

        if converged:
            end_time = time.time()
            observation, info = env.reset()

    duration = 0
    if converged:
        duration = end_time - start_time
        print(str(round(duration, 3)) + " seconds to converge")
    else:
        duration = None
        print("No convergence")
        
    return (currentEpisode-1), duration

In [ ]:
monteCarlo_everyVisit(2000,1)

Episode: 1/2000
Trial Time: 9.84 sec
Q-Table:
0: [0, 0, 0, 0]
1: [0, 0, 0, 0]
2: [0, 0, 0, 0]
3: [0, 0, 0, 0]
4: [0, 0, 0, 0]
5: [0, 0, 0, 0]
6: [0, 0, 0, 0]
7: [0, 0, 0, 0]
8: [0, 0, 0, 0]
9: [0, 0, 0, 0]
10: [0, 0, 0, 0]
11: [0, 0, 0, 0]
12: [0, 0, 0, 0]
13: [0, 0, 0, 0]
14: [0, 0, 0, 0]
15: [0, 0, 0, 0]
16: [0, 0, 0, 0]
17: [0, 0, 0, 0]
18: [0, 0, 0, 0]
19: [0, 0, 0, 0]
20: [0, 0, 0, 0]
21: [0, 0, 0, 0]
22: [0, 0, 0, 0]
23: [0, 0, 0, 0]
24: [0, 0, 0, 0]
25: [0, 0, 0, 0]
26: [0, 0, 0, 0]
27: [0, 0, 0, 0]
28: [0, 0, 0, 0]
29: [0, 0, 0, 0]
30: [0, 0, 0, 0]
31: [0, 0, 0, 0]
32: [0, 0, 0, 0]
33: [0, 0, 0, 0]
34: [0, 0, 0, 0]
35: [0, 0, 0, 0]
36: [0, 0, 0, 0]
37: [0, 0, 0, 0]
38: [0, 0, 0, 0]
39: [0, 0, 0, 0]
40: [0, 0, 0, 0]
41: [0, 0, 0, 0]
42: [0, 0, 0, 0]
43: [0, 0, 0, 0]
44: [0, 0, 0, 0]
45: [0, 0, 0, 0]
46: [0, 0, 0, 0]
47: [0, 0, 0, 0]
change in Q: 0
